# First Simulation for CausalMARL

## TODO
1. debug: 
    1. try DGP with no neigh [identity ADJ]
1. Tuning:
    1. Two penalties in Susan: their CV method
    2. [w_hidden, Learning_rate] of NN: CV
    4. Now: manually fix some values
    3. **Q-V CV**: hard to choose neighbour actions. we can do that? but very large?
5. spatial information - model
2. DR w/o mean-field 
    1. R_it: 
        1. with neigh states/actions as appending
        2. with global states/actions as appending 
    2. \bar{R}_t + global states and actions
    3. multi-dim actions    

1. why wi estimation is not good?
    1. why R2 close to  0? what is our attempt?
    2. density close to zero, den ratio is very large, even after normalization, std is large
1. how to evaluate the true density and Q/V functions?
2. Check the current model and policy
    1. stationary?
1. Design a toy example
    1. need DR (how?)
        1. weight learned with a linear function
        2. Q learned wtih a linear function
    2. need spatial (mean state and action)
        1. R_i = X1 * X2

### Observations

1. when behav = target 
    1. validation is not good  
    2. MSE is low
2. close to behav
    1. 本身就是自然会被带跑的？要比较像 比较好?
    2. std 都不大，基本就靠bias
    3. V / w both can not (eta learn nothing)
    4. 
3. over attraction
    1. do we need the correct reward leads to better performance?
1. explore our model
    1. under behavipour 是stationaty的吗
    2. reward v.s. mismatch? learn something?
1. current results
1. 

### Tried (03/)
1. D_{t+1}
2. increase noise / T 
3. u_O != u_D
4. mean-reversion 在现在的setting下面 又不会有advantage了
7. T_A 三个levels
8. w_i estimation
9. baseline effect
2. interference effect不够强
10. 可以看看那些weight都多大吗
11. decrease order differences
2. strength of attraction: 
1. trivial
    1. burn-in
    1. centeralization of Q
    1. fixed 1/2 in target [tried, not now]
    1. number of neigh
1. mean-reversion

    
## 03/25
1. order size consistency. cutoff ( <1 to < 0)
2. order 的 effect 要小一点
3. / n_neigh
4. standardization

## 03/27
1. 我直接试一试 非spatial的吧
5. check the QV / w by ourself


2. spatial的结构 没有用进去？
3. mean reward useless?
4. learn nothing?


# Setting
### Model
0. Mis
    1. Size: grid, N = 5 * 5, T = 48 * 14
    2. at time t, based on $O_t$ and $D_t$ which are observations  during (t-1, t], we (randomly) choose action $A_t$ which will be implemented during (t, t + 1], and observe the reward $R_t$ which is defind as the mismatch during (t, t + 1].
1. State:
    1. Order: 
        1. $O_{t,g} \sim Poisson(10)$
        2. $O_{t,g} \sim Poisson(10 * (2 - sin(t/48 * 2\pi)))$ [no heterogeneity] -> mean is 20?
    2. Driver: 
        1. Current: $\vec{D}_{t+1} = \omega_t^{-1} \mathbb{P}(((w_O * \mathbf{O}_t + \mathbf{I}) \times (w_A * \mathbf{A}_{t} +  \mathbf{I}) \times \mathbf{Adj}  \times (w_O * \mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + \vec{\epsilon}^D_{t+1})$
            1. $w_t$ is a normalization parameter, $\mathbf{A}_{t}$ and $\mathbf{O}_t$ are corresponding diagonal matries, and $\vec{\epsilon}^D_t \sim (Poisson(1) - 1)$
            2. operator $\mathbb{P}$ is defined as $\mathbb{P}(\vec{D}) = (max(int(D_1),1),\dots, max(int(D_N),1))^T$
        2. $\vec{D}_{t+1} = \frac{1}{2}(\omega_t^{-1} (\mathbf{O}_t \times (\mathbf{A}_{t} + \mathbf{I}) \times \mathbf{Adj}  \times (\mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + 10) + \vec{\epsilon}^D_{t+1}$
        3. $\vec{D}_{t+1} = \frac{1}{2}(\omega_t^{-1} (\mathbf{O}_t \times (\mathbf{A}_{t} + \mathbf{I}) \times \mathbf{Adj}  \times (\mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + 10 * (2 - sin(t/48 * 2\pi))) + \vec{\epsilon}^D_{t+1}$
    4. Mismatch: 
        1. $M_{t+1,g} = 0.5 * (1-\frac{|D_{t+1,g}-O_{t+1,g}|}{|D_{t+1,g}+O_{t+1,g}|}) + 0.5 * M_{t,g}$
        2. $M_{t+1,g} = 1-\frac{|D_{t+1,g}-O_{t+1,g}|}{|D_{t+1,g}+O_{t+1,g}|}$
3. Action: $A_{t,g} \sim Bernoulli(0.5)$
4. Reward: 
    1. current: $R_{t,g} = M_{t + 1, g}min(D_{t + 1,g}, O_{t + 1,g}) + \epsilon^R_{t,g}, \epsilon^R_{t,g} \sim N(0,sd_R)$
    2. $R_{i,t}=M_{i,t} * min(D_{i,t}, O_{i,t})+mean \{M_{j,t} * min(D_{j,t}, O_{j,t})\}_{j\in N(i)}+\epsilon_{i,t}$
5. Target policy: 
    1. $A_{t,g} = a_g, a_g \sim Bernoulli(0.5)$
    2. current: $A_{t,g} = I_{\{u_{O,g} \ge 12\}}$
6. Ts and Ta is the average function.


### Metrics and competing methods
1. Ground truth: MC-based proxy
2. Metrics: bias, std and MSE (100 replicates)
3. Competing methods: 
    1. "IS": Importance Sampling (Lihong Li) + mean-field
    2. "Susan": Value funcions (Susan) + mean-field
    3. "DR_NS": DR without considerinig other agents
    3. DR without mean-field (not yet)
8. Report results for four different $\{a_g\}_g$ patterns  


1. IS:
    1. single agent - bias
    2. spaatial - large variance. [看draft]
2. DR:
    1. better than IS
    
### Time cost

1. l = 5: 9 mins / batch on EC2

### Derivations of DR2

1. DR: $\hat{Q}(\pi(S), S)+I(A=\pi(S))/ b(A|S) (R-\hat{Q}(a,S))$ - (15)
2. IS: $I(A=\pi(S))/ b(A|S) R$
3. Q: $\hat{Q}(\pi(S),S)$
4. 2+3-1: $I(A=\pi(S))/b(A|S) \hat{Q}(a,S)$, it is even not doubly robust

1. Q is correct: Q + IS - DR  -  true = IS - DR = wi(Q - Q)
2. IS is correct: ... = Q - DR = -[w (R+Q - Q - V) ]
3. ours DR: wi(R + Q - Q)
4. ours like them: w_i*R + V - Q - w_i(R + Q' - Q)  = V - Q - w_i(Q' - Q)
4. target: (15)

any weight estimation problem?